In [1]:
# Package Import
import pandas as pd
import numpy as np
import re

# Path
path_raw = r'C:/Users/Usuario/Documents/b4d/data/raw/'
path_processed = r'C:/Users/Usuario/Documents/b4d/data/processed/'

# Name of file
file_name = 'tigobo_connectly_report_v2_052025'

file_name_base = 'tigobo_1804-Internet Inicial 2-Wpp'

# File extension
file_extension = '.csv'

# Suffix to proceseed files
suffix = '_clean'

In [2]:
# Reading reporte csv
# Columns
columns = ['customer_external_id', 'campaign_name', 'dispatched_at', 'sent_at', 'delivered_at', 'read_at', 'error_code', 'error_msg']

# Reading csv to dataframe
df = pd.read_csv(path_raw+file_name+file_extension, usecols=columns)
print(f"Shape:  {df.shape}")

Shape:  (615328, 8)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_4704\1447804892.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_raw+file_name+file_extension, usecols=columns)


In [ ]:
# (Optional)
# Reading base/original csv
# Columns
columns_base = ['PhoneNumber']

# Reading csv to dataframe
df_base = pd.read_csv(path_raw+file_name_base+file_extension, usecols=columns_base)
print(f"Shape:  {df_base.shape}")


In [ ]:
# (Optional)
# Analysis 1.1
# why some phone number was not sent the difussion message
df_diff = df_base[~df_base['PhoneNumber'].isin(df['customer_external_id'])]
print(f"Shape:  {df_diff.shape}")

suffix_diff = '_not_merged'

In [ ]:
# Fix 1.1 - Febrero
# replace name test campaing in order to follow the rule
#df['campaign_name'] = np.where(df['campaign_name']=='Cambio de caracteristicas de planes movil Postpago_Mega Plan 169_21022025', 'Cambio De Caracteristicas De Planes Movil Postpago Mega Plan 169 21022025 . Prueba',df['campaign_name'])
#df.shape

In [ ]:
# Fix 2.1
# Exclude some campaigns beacuse it was error sending
#df = df[df['campaign_name']!='Comunicacion Por Whatsapp Recordatorio Cambio De Caracteristicas Movil Mega Plan 180 Mega Plan 225 Mega Plan 299 Mega Plan 305 . Planes Movil Postpago Mega Plan 299 . 21 02 25 . Lote']
#df.shape

In [3]:
# parameter name campaign
df['campaign_name'] = np.where(
    df['campaign_name'] == 'Comunicacion Whatsapp . Aumento De Velocidad Hogar 01 05 . Full Tigo Plus Tv Medio B2b . 05 05 25 . 16 00 . Prueba copy 1', 
    'Comunicacion Whatsapp . Aumento De Velocidad Hogar 01 05 . Full Tigo Plus Tv Medio B2b . 05 05 25 . 16 00 . Lote 1',
    df['campaign_name']
)

df['campaign_name'] = np.where(
    df['campaign_name'] == 'Comunicacion Whatsapp . Cambio De Caracteristicas Hogar B2b . Internet Empresa 3 B2b . 21 05 25 . 15 30 . Prueba 1 copy 1', 
    'Comunicacion Whatsapp . Cambio De Caracteristicas Hogar B2b . Internet Empresa 3 B2b . 21 05 25 . 15 30 . Lote 1',
    df['campaign_name']
)

df['campaign_name'] = np.where(
    df['campaign_name'].str.upper() == 'COMUNICACION WHATSAPP . CAMBIO DE CARACTERISTICAS HOGAR . WIFI HOGAR ESENCIAL V0 . 19 05 25 . 10 30 . PRUEBA COPY 1',
    'COMUNICACION WHATSAPP . CAMBIO DE CARACTERISTICAS HOGAR . WIFI HOGAR ESENCIAL V0 . 19 05 25 . 10 30 . LOTE 1',
    df['campaign_name']
)

df['campaign_name'] = np.where(
    df['campaign_name'].str.upper() == 'COMUNICACION WHATSAPP . CAMBIO DE CARACTERISTICAS HOGAR B2B . WIFI HOGAR AVANZADO B2B . 21 05 25 . 17 00 . PRUEBA COPY 1',
    'COMUNICACION WHATSAPP . CAMBIO DE CARACTERISTICAS HOGAR B2B . WIFI HOGAR AVANZADO B2B . 21 05 25 . 17 00 . LOTE 1',
    df['campaign_name']
)

In [4]:
# Upper name for all name campaigns
df['campaign_name'] = df['campaign_name'].str.upper()

In [5]:
# Formating the campaign name
# Regexs
#regex_1 = r'\s*?\.?\s*?(BATCH|NORMAL|CX|LOTE|LOT|LO|COPY).*$'
regex_1 = r'\s*?\.?\s*?(BATCH|NORMAL|CX|LOTE|LOT|COPY).*$'
regex_2 = r'\.\s\d{2}$'
regex_3 = r'\s*?\.?\s*?(OK|ENVIO OK|ENVIO|PRUEBA INTERNA|PRUEBA)\s?\d*?$'
regex_4 = r'\.?\s?PRUEBA\s?'

# Function to extraction string with regex
def f_regex(data_str, regex, str_replace=''):
    return re.sub(regex, str_replace, data_str, flags=re.IGNORECASE)

# Applying the regex with the campaing name column
df['campaign_name_base'] = df['campaign_name'].apply(lambda x: f_regex(x, regex_1))
df['campaign_name_base'] = df['campaign_name_base'].apply(lambda x: f_regex(x, regex_2))
df['campaign_name_base'] = df['campaign_name_base'].apply(lambda x: f_regex(x, regex_3))
df['campaign_name_base'] = df['campaign_name_base'].apply(lambda x: f_regex(x, regex_4))

# Verify if this campaign is test
df['tipo'] = np.where(df['campaign_name'].str.contains('PRUEBA', na=False), 'PRUEBA', 'LOTE')

# Filter to verify if this message was error
# This filter was used in excel and below is equivalent with pandas
# =O(ESBLANCO([@[error_code]]);NO(ESBLANCO([@[read_at]]));NO(ESBLANCO([@[delivered_at]]));NO(ESBLANCO([@[sent_at]])))
filter_estado = (
   # df['error_code'].isna() |                  
    df['read_at'].notna() |                    
    df['delivered_at'].notna() 
   # | df['sent_at'].notna()                      
)
df['estado'] = np.where(filter_estado, 'VERDADERO', 'FALSO' )

# old code
#df['estado'] = ~((df['error_code'] == '') | ~(df['error_code'].isna()))
# df.dtypes

In [ ]:
# Opcional
# Exclude some campaigns beacuse it was error sending
df_1 = df[df['campaign_name_base']=='COMUNICACION WHATSAPP CAMBIO DE CARACTERISTICAS HOGAR 01 05 . PLAN INTERNET INICIAL . 18 04 25 . 13 30']
df_1.shape

In [ ]:

df_diff = df_base[df_base['PhoneNumber'].isin(df_1['customer_external_id'])]
print(f"Shape:  {df_diff.shape}")

In [6]:
# Count message for each campaigns
#df.groupby('campaign_name_base', as_index=False).size()
df_tmp = df.copy()
df_tmp['dummy'] = 1
result = pd.crosstab(
        [df_tmp['campaign_name_base']], 
        df_tmp['tipo'], 
        margins=True
    ).reset_index()
result


tipo,campaign_name_base,LOTE,PRUEBA,All
0,COMUNICACION POR WHATSAPP . CLASICO ESPANOL ES...,238943,4,238947
1,COMUNICACION WHATSAPP . AUMENTO DE VELOCIDAD H...,0,3,3
2,COMUNICACION WHATSAPP . AUMENTO DE VELOCIDAD H...,15,0,15
3,COMUNICACION WHATSAPP . AUMENTO DE VELOCIDAD H...,7,3,10
4,COMUNICACION WHATSAPP . CAMBIO DE CARACTERISTI...,29725,5,29730
...,...,...,...,...
56,COMUNICACION WHATSAPP REZAGADOS MOVIL POST PAG...,4,0,4
57,COMUNICACION WHATSAPP REZAGADOS MOVIL POST PAG...,1,0,1
58,COMUNICACION WHATSAPP REZAGADOS MOVIL POST PAG...,1,0,1
59,GESTION DTH . 01 05 25 . TIGO TV PRE PAGO . CL...,17089,0,17089


In [ ]:
# Total by error code
#df.groupby(['error_code', 'error_msg'], as_index=False).size()

df_tmp = df.copy()
df_tmp['dummy'] = 1
result = pd.crosstab(
        [df_tmp['error_code'], df_tmp['error_msg']], 
        df_tmp['dummy'], 
        margins=True
    ).reset_index()
result


In [ ]:
# Total by estado
#df.groupby('estado', as_index=False).size()

df_tmp = df.copy()
df_tmp['dummy'] = 1
result = pd.crosstab([df_tmp['estado']],df_tmp['dummy'],margins=True)
result

In [ ]:
# Change error code
# Its not important
# df['error_code'] = np.where(df['error_code']=='ERROR_CODE_NOT_ELIGIBLE_FOR_DELIVERY',131049,df['error_code'])
# df.shape

In [ ]:
# Format type of data column
df['sent_at'] = pd.to_datetime(df['sent_at'], errors='coerce')
df['dispatched_at'] = pd.to_datetime(df['dispatched_at'], errors='coerce')
df['delivered_at'] = pd.to_datetime(df['delivered_at'], errors='coerce')
df['read_at'] = pd.to_datetime(df['read_at'], errors='coerce')
df['error_code'] = np.floor(pd.to_numeric(df['error_code'], errors='coerce')).astype('Int64') #df['error_code'].astype('Int64')

#df['Tipo'] = np.where(df['tipo'], "PRUEBA", "LOTE")
#df['Estado'] = np.where(df['estado'], "VERDADERO", "FALSO")

In [ ]:
# drop column
df.drop('campaign_name', axis=1, inplace=True)

# reorder columns 
df = df[['customer_external_id', 'campaign_name_base', 'dispatched_at', 'sent_at',
       'delivered_at', 'read_at', 'error_code', 'error_msg', 'estado', 'tipo']]
df.shape

In [ ]:
df.tail(2)

In [ ]:
# format datetime
df['sent_at'] = df['sent_at'].dt.strftime('%d/%m/%Y %H:%M')
df['dispatched_at'] = df['dispatched_at'].dt.strftime('%d/%m/%Y %H:%M')
df['delivered_at'] = df['delivered_at'].dt.strftime('%d/%m/%Y %H:%M')
df['read_at'] = df['read_at'].dt.strftime('%d/%m/%Y %H:%M')

In [ ]:
# Export as csv
df.to_csv(path_processed+file_name+suffix+file_extension)

In [ ]:
# (Opcional)
# Export as csv
df_diff.to_csv(path_processed+file_name_base+suffix_diff+file_extension)